In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf #stock api


In [4]:
#Tickers
start = '2020-01-02'
end = '2021-07-02'

ticker_adjclose = yf.download('MSFT, CAT, AAPL, AMD', period = "ytd", start = start, group_by = 'column',auto_adjust=True, threads = True)['Close']

[*********************100%***********************]  4 of 4 completed


In [ ]:
log_returns = np.log(1+ticker_adjclose.pct_change().dropna())
cov = log_returns.cov() #Covariance matrix
weight = np.full((len(cov),1), 1/len(cov)) #Start from equal weight portfolio
bounds = ((0,1),)*len(cov)

In [34]:
def efficient_portfolio(ticker_adjclose, log_expected_return = 0, short_position = False):
    #Data
    log_returns = np.log(1+ticker_adjclose.pct_change().dropna())
    cov = log_returns.cov() #Covariance matrix
    weight = np.full((len(cov),1), 1/len(cov)) #Start from equal weight portfolio
    bounds = ((0,1),)*len(cov)

    #Function and constrains
    def risk_func(weight):
        return np.dot(weight.T, np.dot(cov,weight))
    
    def Sum_to_one(weight):
        return np.sum(weight)-1

    def Min_return(weight):
        return log_expected_return - np.sum(log_returns.mean() * weight)
    
    constraints = ({'type': 'eq', 'fun': Sum_to_one},{'type': 'eq', 'fun': Min_return})

    if log_expected_return > 0 :
        solver_e = minimize(risk_func, weight, method = 'SLSQP', bounds = bounds*short_position, constraints = constraints)
        weight = solver_e.x
    else:
        solver_v = minimize(risk_func, weight, method = 'SLSQP', bounds = bounds*short_position, constraints = constraints[0])
        weight = solver_v.x
    
    return weight

In [39]:
print(efficient_portfolio(ticker_adjclose, 0.001, True))

[0.1655059  0.06925476 0.54716828 0.21807107]


In [ ]:
#wararnty

def warranty(weights):
    w_weights = weights / sum(abs(weights))
    warranty = sum([neg for neg in w_weights if neg < 0]) * -2
    return w_weights, warranty

In [27]:

#risk free asset
log_risk_free_return = 0.0006
log_expected_return = 0.001
log_expected_volatility = 0.01

cov_inverse = np.linalg.inv(cov)
ret_vector = log_returns.mean() - log_risk_free_return

pr_weights_z = np.dot(cov_inverse,ret_vector)
pr_weights_xm = pr_weights_z / (sum(pr_weights_z))

w_weights, warranty_w = warranty(pr_weights_xm)

portfolio_risky_mean = np.dot(log_returns.mean().T,w_weights) + warranty_w * log_risk_free_return
portfolio_risky_var = np.dot(w_weights.T,np.dot(cov, w_weights))

#Expected return

portfolio_risky_weight_e = (log_expected_return - log_risk_free_return) / (portfolio_risky_mean - log_risk_free_return)

#Expected volatility

portfolio_risky_weight_v = log_expected_volatility / np.sqrt(portfolio_risky_var)

#Total mean

portfolio_mean = portfolio_risky_weight_e * portfolio_risky_mean + (1-portfolio_risky_weight_e) * log_risk_free_return

In [ ]:
#Solving by hand
ret_mean = log_returns.mean()
ret_exp = 0.001

def variance_minimizer(ret_exp, log_returns, cov):
    ret_mean = log_returns.mean()
    w_con = np.ones(len(cov)) #weight constraints
    cov_inverse = np.linalg.inv(cov) #Inverse matrix

    #Matrix multiplications (so the code looks cleaner)
    cov_cons_mean = np.dot(ret_mean.T, np.dot(cov_inverse, ret_mean))
    cov_cons_weight = np.dot(w_con.T, np.dot(cov_inverse, w_con))
    cov_cons_wm = np.dot(ret_mean.T, np.dot(cov_inverse, w_con))

    return (1/(cov_cons_mean * cov_cons_weight - cov_cons_wm**2)) * np.dot(cov_inverse,((cov_cons_weight*ret_exp - cov_cons_wm) * ret_mean + (cov_cons_mean - cov_cons_wm*ret_exp) * w_con))

variance_minimizer(ret_exp, ret_mean, cov)

In [ ]:
variance_minimizer(ret_exp, ret_mean, cov)

array([ 3.76389943,  2.21654257, -5.58202935,  0.60158735])